In [1]:
%load_ext autoreload
%autoreload 2

# Imports

In [2]:
import os
import pandas as pd

import numpy as np
import jax
jax.config.update('jax_platform_name', 'gpu')



In [3]:
fn_EcoCyc = '../data/sRNA/EcoCyc/EcoCyc_regulatory_RNAs.csv'
fn_RNAInter = os.path.join('..', 'data', 'sRNA', 'RNAInter', 'Download_data_RR.csv')
fn_srnatarbase = os.path.join('..', 'data', 'sRNA', 'sRNATarBase', 'sRNATarBase.csv')
ecocyc = pd.read_csv(fn_EcoCyc)
rnainter = pd.read_csv(fn_RNAInter)
srnatarbase = pd.read_csv(fn_srnatarbase)
# data = pd.concat([pd.read_csv(fn_RNAInter), pd.read_csv(fn_EcoCyc)])
try:
    rnainter = rnainter.drop(columns=['Unnamed: 0'])
    srnatarbase = srnatarbase.drop(columns=['Unnamed: 0'])
except:
    pass


### Start with RNAInter

In [4]:
np.sum(rnainter['Sequence1'].isna()) + np.sum(rnainter['Sequence2'].isna())

2

In [5]:
rnainter

,level_0,index,RNAInterID,Interactor1.Symbol,Category1,Species1,Interactor2.Symbol,Category2,Species2,Raw_ID1,Raw_ID2,score,strong,weak,predict,Sequence1,Sequence2
0,0,1029724,RR05384747,acnA,mRNA,Escherichia coli str. K-12 substr. MG1655,ryhB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:946724,NCBI:2847761,0.2292,Northern blot//Reporter assay,NaN,NaN,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...
1,1,1029725,RR05384845,acrZ,mRNA,Escherichia coli str. K-12 substr. MG1655,omrB,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:945365,NCBI:2847747,0.1778,Northern blot,Microarray,NaN,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,TGTTCTATACTTGGGTTCGACTTGGGTTAGACTTGTCTTTACTGTC...
2,2,1029732,RR05387056,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,flhD,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:945442,0.2314,Northern blot//Reporter assay,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,CTCGTGCCGAATTCGGCACGAGCGATATTTCATCAGTTATCGGTAA...
3,3,1029733,RR05387057,arcZ,sRNA,Escherichia coli str. K-12 substr. MG1655,rpoS,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847690,NCBI:947210,0.1996,Northern blot//RACE//RT-PCR//Beta-galactosidas...,NaN,NaN,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,ATCCTCGGGTCTTGCAGGCCACACAGGACACCCTGAACCGTCATGG...
4,4,1029734,RR05387126,argR,mRNA,Escherichia coli str. K-12 substr. MG1655,dsrA,sRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:947861,NCBI:946470,0.2139,Primer extension assay,NaN,NaN,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,TGAGCAACTTTATTCACATAATTTCTACACCAAGAACTCGAGGTTA...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
349,349,8031771,RR05460721,ryhB,sRNA,Escherichia coli str. K-12 substr. MG1655,ynfF,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847761,NCBI:945268,0.1778,Northern blot,Microarray,NaN,TTTGAGAGCGTCTCTGTCCCTCGTTTTGCGGTTAAGCCGCATCCAT...,ACTTACTACTGTGCCTTTAGGGTGGGGGGATACACCGTTTCCGCCG...
350,350,8031772,RR05460722,sdsR,sRNA,Escherichia coli str. K-12 substr. MG1655,tolC,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:2847772,NCBI:947521,0.3292,Northern blot//Western blot,NaN,NaN,CTACCTCAACACCCAACAATCGTTGATTGAAAACATCTTCTATCGC...,ATGAAGAAATTGCTCCCCATTCTTATCGGCCTGAGCCTTTCTGGGT...
351,351,8031773,RR05460723,sgrS,sRNA,Escherichia coli str. K-12 substr. MG1655,sodB,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:4056038,NCBI:944953,0.1983,Northern blot,NaN,NaN,CCAGAACATTGGCTTTCAAAAGTTTTTTAATTGATGGAAACTCTTT...,GAAGAGAAAAGAAAAAAAGACCTTATTTGCTTATCCTGCATCTCTC...
352,352,8031774,RR05460724,sgrS,sRNA,Escherichia coli str. K-12 substr. MG1655,yigL,mRNA,Escherichia coli str. K-12 substr. MG1655,NCBI:4056038,NCBI:2847768,0.3273,Northern blot//Reporter assay,NaN,NaN,CCAGAACATTGGCTTTCAAAAGTTTTTTAATTGATGGAAACTCTTT...,TGCCGCTGACGAGGGAGAGGATGAGGATTGCGAGCCAGAGCATCCA...


In [6]:
data = pd.DataFrame()
rnainter['Database'] = 'RNAInter'
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor1.Symbol', 'Raw_ID1', 'Category1', 'Interactor1.Symbol', 'Sequence1', 'Database']):
    data[n] = rnainter.drop_duplicates(subset=['Interactor1.Symbol'])[o]
data2 = pd.DataFrame()
for n, o in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence', 'Database'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2', 'Database']):
    data2[n] = rnainter.drop_duplicates(subset=['Interactor2.Symbol'])[o]

# data = pd.concat([data, rnainter.drop_duplicates(subset=['Interactor2.Symbol']).reset_index()[['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Sequence2']].rename(
#     columns={j: i for i, j in zip(['Name', 'ID', 'Category', 'Symbol', 'Sequence'], ['Interactor2.Symbol', 'Raw_ID2', 'Category2', 'Interactor2.Symbol', 'Sequence2'])})], axis=0)
data = pd.concat([data, data2])
data['Alias'] = ''

data = data[~data['Name'].isna()]
data = data.drop_duplicates(subset=['Name'])

In [7]:
data['Symbol'].value_counts()


Symbol
acnA    1
phoP    1
sibC    1
sibB    1
sibA    1
       ..
luxS    1
malK    1
manX    1
manY    1
symR    1
Name: count, Length: 276, dtype: int64

### Now EcoCyc

In [8]:
ecocyc.head()

,Unnamed: 0,Ontology - all instances of class from My Favorites,Matches,Common-Name,GO-Terms,Names,Object ID,Gene,Sequence - DNA sequence,Right-End-Position,Left-End-Position,Product,Accession-1,Accession-2
0,0,6S-RNA,Regulatory-RNAs,6S RNA,NaN,6S RNA // SsrS // Ssr,6S-RNA,EG30099,ATTTCTCTGAGATGTTCGCAAGCGGGCCAGTCCCCTGAGCCGATAT...,3056165,3055983,6S-RNA,b2911,ECK2906
1,1,RDLD-RNA,Regulatory-RNAs,antisense regulatory RNA RdlD,GO:2000766 // GO:0030371,antisense regulatory RNA RdlD // RdlD // RdlD ...,RDLD-RNA,G0-9042,GTCTAGAGTCAAGATTAGCCCCCGTGGTGTTGTCAGGTGCATACCT...,3700199,3700136,RDLD-RNA,b4454,ECK3525
2,2,RNA0-441,Regulatory-RNAs,arfA-derived small regulatory RNA,GO:0006970 // GO:0003729,<i>arfA</i>-derived small regulatory RNA,RNA0-441,G0-17130,AAGAAAGTAAATCACTTTTTTACCACTGGTCTTCTGCTTTCAGGCG...,3438484,3438431,RNA0-441,b4846,ECK4703
3,3,RNA0-409,Regulatory-RNAs,e14 prophage; small regulatory RNA C0293,GO:0035194 // GO:0040033 // GO:0003729,e14 prophage; small regulatory RNA C0293 // C0293,RNA0-409,G0-8897,TACTCACCCGGGACTCGCCAGGGGACAGCCAACAGGCATTGGGTGC...,1196782,1196711,RNA0-409,b4806,ECK4663
4,4,RNA0-162,Regulatory-RNAs,putative antisense regulatory RNA RdlB,GO:1902201,putative antisense regulatory RNA RdlB // RdlB,RNA0-162,G0-9606,GTCTGGTTTCAAGATTAGCCCCCGTTCTGTTGTCAGGTTTTACCTC...,1269923,1269858,RNA0-162,b4422,ECK1207


In [9]:
ecocyc['Category'] = 'sRNA'
ecocyc['Database'] = 'EcoCyc'
ecocyc = ecocyc.drop_duplicates(subset=['Common-Name'])

ecocyc['Symbol'] = ecocyc['Common-Name']
data = pd.concat([data, ecocyc.rename(columns={j: i for i, j in zip(['Name', 'ID', 'Sequence', 'Alias'], ['Common-Name', 'Object ID', 'Sequence - DNA sequence', 'Names'])})[
    ['Name', 'ID', 'Category', 'Symbol', 'Alias', 'Sequence', 'Database']]], axis=0)

### Finally sRNATarBase

In [10]:
srnatarbase

,Target,sRNA,Target Alias,sRNA Alias,Target ID,sRNA ID,Regulation,Target Type,sRNA Type,Target Binding Position,sRNA Binding Position,Target Sequence,sRNA Sequence
0,hns,dsrA,B1; bglY; cur; drc; drdX; drs; ECK1232; fimG; ...,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:945829","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,"['7..19', 'NA', '7..19; 401..411']","['31..43', 'NA', '31..54']",ATGAGCGAAGCACTTAAAATTCTGAACAACATCCGTACTCTTCGTG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
1,rbsD,dsrA,ECK3742; JW5857; rbsP,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:948267","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGAAAAAAGGCACCGTTCTTAATTCTGATATTTCATCGGTGATCT...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
2,argR,dsrA,ECK3226; JW3206; Rarg; xerA,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:947861","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGCGAAGCTCGGCTAAGCAAGAAGAACTAGTTAAAGCATTTAAAG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
3,ilvI,dsrA,ECK0079; JW0076,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:948793","chromosome:NC_000913.3, Gene ID:946470",Repression,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGGAGATGTTGTCTGGAGCCGAGATGGTCGTCCGATCGCTTATCG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
4,rpoS,dsrA,abrD; appR; csi2; dpeB; ECK2736; JW5437; katF;...,ECK1952; IS095; JWR0036,"chromosome:NC_000913.3, Gene ID:947210","chromosome:NC_000913.3, Gene ID:946470",Induction,mRNA,trans-encoded antisense RNA,"['-119..-97', 'NA', 'NA']","['10..32', 'NA', 'NA']",ATGAGTCAGAATACGCTGAAAGTTCATGATTTAAATGAAGATGCGG...,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
404,msrA,ryhB,ECK4215; JW4178; pms; pmsR,ECK3426; IS176; JWR0219; psrA18; sraI,"chromosome:NC_000913.3, Gene ID:948734","chromosome:NC_000913.3, Gene ID:2847761",No Interaction,mRNA,NaN,['NA'],['NA'],ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,GCGATCAGGAAGACCCTCGCGGAGAACCTGAAAGCACGACATTGCT...
405,hdeA,gcvB,ECK3494; JW3478; yhhC; yhiB,ECK2804; IS145; JWR0247; psrA11,"chromosome:NC_000913.3, Gene ID:948025","chromosome:NC_000913.3, Gene ID:2847720",Induction,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...
406,hdeB,gcvB,ECK3493; JW5669; yhhD; yhiC,ECK2804; IS145; JWR0247; psrA11,"chromosome:NC_000913.3, Gene ID:948026","chromosome:NC_000913.3, Gene ID:2847720",Induction,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,ACTTCCTGAGCCGGAACGAAAAGTTTTATCGGAATGCGTGTTCTGG...
407,fliC,Esr41,ECs2662,NaN,"chromosome:NC_002695.1, Gene ID:","chromosome:NC_002695.1, Gene ID:",Induction,mRNA,trans-encoded antisense RNA,['NA'],['NA'],ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,GATGCTCTAGGCATCACATTTTCTCCATGGGGTATTCCCTCCGCCG...


In [11]:
srnatarbase_merged = pd.DataFrame()
srnatarbase_merged['Name'] = np.concatenate(
    [srnatarbase['sRNA'].unique(), srnatarbase['Target'].unique()], axis=0)
srnatarbase_merged['ID'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA ID'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target ID'].iloc[0])
srnatarbase_merged['Alias'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA Alias'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Alias'].iloc[0])
srnatarbase_merged['Category'] = srnatarbase_merged['Name'].apply(
    lambda x: 'sRNA' if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Type'].iloc[0])
srnatarbase_merged['Symbol'] = srnatarbase_merged['Name']
srnatarbase_merged['Sequence'] = srnatarbase_merged['Name'].apply(
    lambda x: srnatarbase[srnatarbase['sRNA'] == x]['sRNA Sequence'].iloc[0] if x in srnatarbase['sRNA'].tolist()
    else srnatarbase[srnatarbase['Target'] == x]['Target Sequence'].iloc[0])
srnatarbase_merged['Database'] = 'sRNATarBase'
srnatarbase_merged

,Name,ID,Alias,Category,Symbol,Sequence,Database
0,dsrA,"chromosome:NC_000913.3, Gene ID:946470",ECK1952; IS095; JWR0036,sRNA,dsrA,AACACATCAGATTTCCTGGTGTAACGAATTTTTTAAGTGCTTCTTG...,sRNATarBase
1,micF,"chromosome:NC_000913.3, Gene ID:2847742",ECK2208; IS113; JWR0043; stc,sRNA,micF,GCTATCATCATTAACTTTATTTATTACCGTCATTCATTTCTGAATG...,sRNATarBase
2,micC,"chromosome:NC_000913.3, Gene ID:2847713",ECK1373; IS063; JWR0234; tke8,sRNA,micC,GTTATATGCCTTTATTGTCACAGATTTTATTTTCTGTTGGGCCATT...,sRNATarBase
3,oxyS,"chromosome:NC_000913.3, Gene ID:2847701",ECK3952; JWR0098,sRNA,oxyS,GAAACGGAGCGGCACCTCTTTTAACCCTTGAAGTCACTGCCCGTTT...,sRNATarBase
4,istR,"chromosome:NC_000913.3, Gene ID:5061525",ECK4425; psr19; psrA19,sRNA,istR,GCACTAAATACGTCAAAATTCGTGCCGAAATTGCGCGTTCTGCGCG...,sRNATarBase
...,...,...,...,...,...,...,...
319,msrA,"chromosome:NC_000913.3, Gene ID:948734",ECK4215; JW4178; pms; pmsR,mRNA,msrA,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase
320,hdeA,"chromosome:NC_000913.3, Gene ID:948025",ECK3494; JW3478; yhhC; yhiB,mRNA,hdeA,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase
321,hdeB,"chromosome:NC_000913.3, Gene ID:948026",ECK3493; JW5669; yhhD; yhiC,mRNA,hdeB,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase
322,fliC,"chromosome:NC_002695.1, Gene ID:",ECs2662,mRNA,fliC,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase


In [12]:
data = pd.concat([data, srnatarbase_merged], axis=0)
data = data.reset_index().drop(columns='index')

# Delete duplicates responsibly

In [13]:
doubles = data['Symbol'].value_counts()[data['Symbol'].value_counts() > 1].index.to_list()
len(doubles)

267

In [15]:
for d in doubles:
    
    if len(data[data['Symbol'] == d]['Sequence'].unique()) == 1:
        print('Removing duplicate of ', d, data[data['Symbol'] == d]['Name'].unique())
        data = data.drop(index=data[(data['Symbol'] == d) & (data['Database'] != 'EcoCyc')].index[0])
    else:
        print('Did not remove', d)
        # print(data[data['Symbol'] == d]['Sequence'].values)
    
data = data[~data['Sequence'].isna()]
data
# data[data['Symbol'] == d].index[0]

Did not remove acnA
Did not remove phoP
Did not remove sibC
Did not remove sibB
Did not remove sibA
Did not remove sokC
Did not remove sokB
Removing duplicate of  sstT ['sstT']
Removing duplicate of  oppA ['oppA']
Did not remove omrB
Removing duplicate of  hdeB ['hdeB']
Removing duplicate of  hdeA ['hdeA']
Removing duplicate of  yobA ['yobA']
Did not remove sodB
Removing duplicate of  sodA ['sodA']
Removing duplicate of  narK ['narK']
Did not remove sibD
Did not remove sibE
Did not remove tisB
Did not remove rdlA
Did not remove rdlB
Did not remove rdlC
Did not remove rdlD
Did not remove ygdQ
Removing duplicate of  pal ['pal']
Did not remove tsx
Did not remove ybgF
Did not remove yfeK
Removing duplicate of  ygiM ['ygiM']
Did not remove yneM
Removing duplicate of  phoE ['phoE']
Did not remove acrZ
Did not remove rseX
Removing duplicate of  metE ['metE']
Did not remove maeA
Did not remove iscR
Removing duplicate of  symE ['symE']
Did not remove paoA
Did not remove pinH
Did not remove proA

,Name,ID,Category,Symbol,Sequence,Database,Alias
0,acnA,NCBI:946724,mRNA,acnA,ATTCGGAACGAGGCCTGAAGCAGTGTCGCCGTCCCTCTGCCTTGCA...,RNAInter,
1,acrZ,NCBI:945365,mRNA,acrZ,ACTTACTACTGTCTTCGGGGGGTCCGAGGTTTCTGGGGGGTCGTAC...,RNAInter,
2,arcZ,NCBI:2847690,sRNA,arcZ,CTCATGTTGACCGCTTGTTTAGCAGCTTCAAGGAAGCTGAAGGGCA...,RNAInter,
3,argR,NCBI:947861,mRNA,argR,GACAATGGCGATAGTATGGCGGTTGTTTCTTTCCCATCTCTACTCA...,RNAInter,
4,arrS,NCBI:11115378,sRNA,arrS,CTCTCCCTCTCTTTCTCTCTTCTCCGCGGCGATACGATCCGAGATG...,RNAInter,
...,...,...,...,...,...,...,...
670,msrA,"chromosome:NC_000913.3, Gene ID:948734",mRNA,msrA,ATGAGTTTATTTGATAAAAAGCATCTGGTTTCCCCCGCCGATGCCC...,sRNATarBase,ECK4215; JW4178; pms; pmsR
671,hdeA,"chromosome:NC_000913.3, Gene ID:948025",mRNA,hdeA,ATGAAAAAAGTATTAGGCGTTATTCTTGGTGGTCTGCTTCTTCTGC...,sRNATarBase,ECK3494; JW3478; yhhC; yhiB
672,hdeB,"chromosome:NC_000913.3, Gene ID:948026",mRNA,hdeB,ATGAATATTTCATCTCTCCGTAAAGCGTTTATTTTTATGGGCGCTG...,sRNATarBase,ECK3493; JW5669; yhhD; yhiC
673,fliC,"chromosome:NC_002695.1, Gene ID:",mRNA,fliC,ATGGCACAAGTCATTAATACCAACAGCCTCTCGCTGATCACTCAAA...,sRNATarBase,ECs2662


In [16]:
data.to_csv('../data/sRNA/merged_EcoCyc_RNAInter_sRNATarBase.csv')